In [ ]:
# Import libraries
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
sia = SentimentIntensityAnalyzer()

VADER (Valence Aware Dictionary and sEntiment Reasoner) is effective for analyzing sentiments expressed in social media and short text. It provides a compound score that reflects the overall sentiment.

In [ ]:
# Apply sentiment analysis
df['sentiment'] = df['text'].apply(lambda x: sia.polarity_scores(x))

# Extract compound score and determine sentiment category
df['compound'] = df['sentiment'].apply(lambda x: x['compound'])
df['sentiment_category'] = df['compound'].apply(lambda x: 'positive' if x > 0.05 else ('negative' if x < -0.05 else 'neutral'))

# Display results
print(df[['text', 'sentiment', 'compound', 'sentiment_category']])